# Store

> Provides storage for documents

In [ ]:
#| default_exp store

In [ ]:
#| export
from nanorag.base import Document, abstractmethod, ABC
from nanorag.context import ModelContext
from nanorag.loaders import PDFLoader
from typing import Union, List, Dict, Optional
from uuid import UUID
from collections import defaultdict
from psycopg2.extensions import AsIs
import json


In [ ]:
#| export
class BaseDocumentStore(ABC):
    """
    Base class for document storage"""
    def __init__(self, documents : Dict[str, Document] = {}):
        pass
    @abstractmethod    
    def add(self, document: Document):
        pass
    @abstractmethod 
    def ids(self):
        pass
    @abstractmethod    
    def delete(self, ids: Union[List, str]):
        pass
    @abstractmethod
    def get(self, ids: Union[UUID, List]):
        pass
        

In [ ]:
#| export
class DocumentStore(BaseDocumentStore):
    """Key value type document store that store documents by their id in a dictionary.
    Also checks for duplicates via hashing and doesn't admit them. Compatible with both nodes and documents."""
    def __init__(self, documents : Union[List[Document], Document]= []):
        if isinstance(documents, list):
            self.documents = {document.id: document for document in documents}
        elif isinstance(documents, Document):
            self.documents = {documents.id: documents}

        
    def add(self, document: Union[List[Document], Document]) -> str:
        doc_ids = []
        if isinstance(document, list):
            for doc in document:
                self.add(doc)
                doc_ids.append(doc.id)
        else:
            for stored_document in self.documents:
                if self.documents[stored_document].hash == document.hash:
                    return f"You tried to add a duplicate document: {document.hash}"
                elif self.documents[stored_document].id == document.id:
                    self.documents[document.id] = document
                    return f"Document with id {document.id} has been updated"
            self.documents[document.id] =  document
        return f"The following documents have been added: {doc_ids}"

    def ids(self):
        doc_ids = [doc for doc in self.documents]
        return doc_ids

    def delete(self, ids: Union[List, str]):
        deleted_docs = []
        if isinstance(ids, List):
            for id in ids:
                deleted_doc = self.documents.pop(id, None)
                if deleted_doc != None:
                    deleted_docs.append(deleted_doc)
        elif isinstance(ids, UUID):
            deleted_doc = self.documents.pop(ids, None)
            print(f'Ids are: {ids} and theoretically deleted doc is {deleted_doc}')
            if deleted_doc != None:
                deleted_docs.append(deleted_doc)
        return f"The following docs have been deleted {deleted_docs}"
        
    def get(self, ids: Optional[Union[List[UUID], UUID]] = None) -> Optional[Union[Document, List[Document]]]:
        if ids == None:
            ids = self.ids()
            if isinstance(ids, List):
                if len(ids) == 0:
                    return None
        if isinstance(ids, List):
            docs = [self.documents[id] for id in ids if id in self.documents]
            if len(docs) == 0:
                return None
            return docs
        elif isinstance(ids, UUID):
            doc = self.documents.get(ids, None)
            if doc is None:
                return None
            return doc
        return None
    def group_by_source_id(self, source_id = None): #Other type of filters can be added
        grouped_documents = defaultdict(list)
        for doc in self.documents.values():
            if source_id == None:
                grouped_documents[doc.source_id].append(doc)
            elif doc.source_id not in grouped_documents or doc.source_id == source_id:
                grouped_documents[doc.source_id].append(doc)
        if source_id != None:
            return grouped_documents[source_id]
        return dict(grouped_documents)
#NOTE: Could I store both nodes and docs in same store? 

In [ ]:
#| export
import psycopg2
import os
db_uri = os.environ.get('POSTGRES_URI', None) 

In [ ]:
#| export

class PostgresDocumentStore(BaseDocumentStore):
    
    def __init__(self,db_uri, table_name = 'documents'):
        self.table_name = table_name
        self.conn = psycopg2.connect(db_uri)
        self.cur = self.conn.cursor()
        self.__create_if_not_exists()

    def add(self, documents: Union[List[Document], Document]):

        if isinstance(documents, Document):
            documents = [documents]

        if isinstance(documents, list):
            docs_to_insert = [(str(doc.id), str(doc.source_id), doc.name, doc.text, json.dumps(doc.metadata), doc.hash, doc.metadata.get('category', 'UNCATEGORIZED')) for doc in documents]
            try:
                self.cur.executemany(f"""
                INSERT INTO {self.table_name} (id, source_id, name, text, metadata, hash, category)
                VALUES (%s, %s, %s, %s, %s, %s, %s);
                """,docs_to_insert)
                self.conn.commit()
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                return None
        doc_ids = [doc[0] for doc in docs_to_insert]
        return f"The following documents have been added: {doc_ids}"
        #For now not including any relationship
    def ids(self):
        pass
    def delete(self):
        pass
    def get(self):
        pass


    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()

    def __create_if_not_exists(self, schema_name = 'public'):
        """
        Creates a table for storing documents if it doesn't exist in the database,
        returns True if the table already exists and False if it was just created. 
        """
        try:
            self.cur.execute(f"""
                SELECT EXISTS (
                SELECT FROM pg_tables
                WHERE  schemaname = '{schema_name}'
                AND    tablename  = '{self.table_name}'
                );
                """)

            table_exists = self.cur.fetchone()[0]

            if not table_exists:
                print('Does not exist')
                self.cur.execute(f"""
                CREATE TABLE {self.table_name} (
                    id UUID PRIMARY KEY,
                    source_id UUID NOT NULL,
                    name TEXT,
                    text TEXT NOT NULL,
                    metadata JSONB,
                    hash TEXT UNIQUE NOT NULL,
                    prev_node UUID,
                    next_node UUID,
                    category VARCHAR(255),
                    FOREIGN KEY (prev_node) REFERENCES {self.table_name}(id),
                    FOREIGN KEY (next_node) REFERENCES {self.table_name}(id));
                """)
                self.conn.commit()
                return False
            else:
                return True
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return None
        
    def close(self):
        if self.cur is not None:
            self.cur.close()
        if self.conn is not None:
            self.conn.close()


In [ ]:
#| hide 
#| eval: false
loader = PDFLoader('datasets/papers_pdf')
documents = loader.get_documents()
store = PostgresDocumentStore(db_uri)

In [ ]:
#| hide 
#| eval: false
import os
import psycopg2
db_uri = os.environ['POSTGRES_URI']
conn = psycopg2.connect(db_uri)
cur = conn.cursor()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

Example insert into DocStore

In [ ]:
#| hide
#| eval: false
loader = PDFLoader('datasets/papers_pdf')
documents = loader.get_documents()
store = DocumentStore(documents)

In [ ]:
#| hide
#| eval: false
documents_2 = loader.get_documents()
store.add(documents_2)
source_id = documents_2[0].source_id
document = store.group_by_source_id(source_id)
document